**FOR LEVEL DAMAGE**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**IMAGE PRE PROCESSING**

**1. Import The ImageDataGenerator Library**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [ ]:
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Car damage/level/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/Car damage/level/validation',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

**MODEL BUILDING**

**1. Importing The Model Building Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/gdrive/MyDrive/Car damage/level/training'
valid_path = '/content/gdrive/MyDrive/Car damage/level/validation'

In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

**3. Adding Flatten Layer**

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/gdrive/MyDrive/Car damage/level/training/*')

In [ ]:
folders

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
len(folders)

**4. Adding Output Layer**

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [ ]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.summary()

**6. Configure The Learning Process**

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

**8. Save The Model**

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/gdrive/MyDrive/Car damage/level.h5')

**9. Test The Model**

In [ ]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [ ]:
model = load_model('/content/gdrive/MyDrive/Car damage/level.h5')

In [ ]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [ ]:
import numpy as np

In [ ]:
data = "/content/gdrive/MyDrive/Car damage/level/validation/01-minor/0005.JPEG"
image = cv2.imread(data)
print(detect(image))